# Responsável por Importar Contratos do Login

## Importando bibliotecas

In [41]:
import sys, os

# Carregando pacote que estao fora de /transform
path_raiz = os.path.abspath(os.path.join("..", ".."))
sys.path.append(path_raiz)

In [42]:
import pandas as pd
from src.utils import carregar_tabela_excel
from src.insert.insert_login import inserir_radusuarios
import json
import datetime

## Importando e tratando os dados

In [43]:
# Obter a data e hora atual
now = datetime.datetime.now()

# Formatar a data e hora como 'YYYYMMDD_HHMMSS'
timestamp = now.strftime("%Y%m%d_%H%M%S")
timestamp

'20250319_230627'

In [44]:
source_input = "../../data/baseTesteIXC_Transform.xlsx"
source_input_contrato_insertd = "../../src/inserted/contrato_insert_sucess.xlsx"
source_output = f"../../src/inserted/login_insert_sucess.xlsx"

In [45]:
# Importa tabela de cliente para puxar o id do cliente
df_contrato = pd.read_excel(source_input_contrato_insertd, dtype=str).fillna("")
df_contrato

,Unnamed: 0,idClienteIXC,filial_id,idCidade,Endereco,Numero,Bairro,Cep,idPlanoIXC,idVencimentoIXC,Login,Senha,nomePlano,idContratoIXC,statusInsercao,logRetorno
0,0,117696,35,1659,Vila São Sebastião,SN,Centro,57830-000,6824,25,teste1@radios.com,peixoto,100MB_SPEEDY,154047,sucesso,"{'type': 'success', 'message': 'Registro inser..."


In [46]:
print(df_contrato.isnull().sum())

Unnamed: 0         0
idClienteIXC       0
filial_id          0
idCidade           0
Endereco           0
Numero             0
Bairro             0
Cep                0
idPlanoIXC         0
idVencimentoIXC    0
Login              0
Senha              0
nomePlano          0
idContratoIXC      0
statusInsercao     0
logRetorno         0
dtype: int64


### Adiciona o idGrupo ao dataframe

In [47]:
import os
import sys

# Carregando pacote que estao fora de /transform
path_raiz = os.path.abspath(os.path.join("..", ".."))
sys.path.append(path_raiz)

from config.database_select import executa_query

#### Cria função somente para isso

In [48]:
# Função para adicionar a coluna 'idGrupo' ao DataFrame
def add_idGrupo_to_df_contrato(df_contrato):
    idGrupo_list = []
    for _, row in df_contrato.iterrows():
        idPlanoIXC = row['idPlanoIXC']
        # Monta a query utilizando o idPlanoIXC da linha
        query = f"""
        SELECT
            vdcp.id_plano
        FROM
            vd_contratos vdc
            LEFT JOIN vd_contratos_produtos vdcp ON vdc.id = vdcp.id_vd_contrato AND vdcp.tipo = "I"
        WHERE 
            vdc.id = {idPlanoIXC}
        """
        resultado = executa_query(query)
        # Caso a query retorne resultados, usamos o primeiro valor
        if resultado and len(resultado) > 0:
            id_plano = resultado[0][0]
        else:
            id_plano = None
        idGrupo_list.append(id_plano)
    
    # Cria a nova coluna 'idGrupo' no DataFrame
    df_contrato['idGrupo'] = idGrupo_list
    return df_contrato

In [49]:
df_contrato = add_idGrupo_to_df_contrato(df_contrato)
df_contrato

,Unnamed: 0,idClienteIXC,filial_id,idCidade,Endereco,Numero,Bairro,Cep,idPlanoIXC,idVencimentoIXC,Login,Senha,nomePlano,idContratoIXC,statusInsercao,logRetorno,idGrupo
0,0,117696,35,1659,Vila São Sebastião,SN,Centro,57830-000,6824,25,teste1@radios.com,peixoto,100MB_SPEEDY,154047,sucesso,"{'type': 'success', 'message': 'Registro inser...",6668


### Inserção teste

In [50]:
# inserir_radusuarios(
#     id_cliente="117695",
#     id_contrato="154045",
#     id_grupo="6668",
#     login="weslley123ss",
#     senha="testesenhappoe",
# )

## Insere Login

In [51]:
# Faça uma cópia do DataFrame para evitar o SettingWithCopyWarning
df_contrato = df_contrato.copy()

# Crie as colunas adicionais para registrar o retorno da API (inicialmente vazias)
df_contrato.loc[:, "idRadUsuarios"] = None
df_contrato.loc[:, "statusInsercaoRad"] = None
df_contrato.loc[:, "logRetornoRad"] = None

for idx, row in df_contrato.iterrows():
    # Mapeamento dos dados do DataFrame para os parâmetros da função inserir_radusuarios.
    # Certifique-se de que os campos obrigatórios estejam preenchidos.
    id_cliente = row["idClienteIXC"]      # Valor do cliente
    id_contrato = row["idContratoIXC"]      # Deve vir de uma inserção anterior
    # Como o id_grupo não existe no DataFrame, definimos um valor fixo ou calculado conforme sua regra
    id_grupo = [row["idGrupo"]]  # Exemplo: "6668"
    login = row["Login"]
    senha = row["Senha"]
    cidade = row["idCidade"]
    endereco = row["Endereco"]
    numero = row["Numero"]
    bairro = row["Bairro"]
    cep = row["Cep"]
    # Se necessário, pode-se ajustar senha_md5 ou outros parâmetros, mas aqui usaremos os defaults da função.

    # Chama a função inserir_radusuarios com os parâmetros mapeados
    response = inserir_radusuarios(
        id_cliente=id_cliente,
        id_contrato=id_contrato,
        id_grupo=id_grupo,
        login=login,
        senha=senha,
        cep=cep,
        endereco=endereco,
        numero=numero,
        bairro=bairro,
        cidade=cidade,
        # Os demais parâmetros serão utilizados com seus valores default definidos na função.
    )

    status_code = response.status_code
    content = response.content  # bytes

    # Faz o parse do JSON retornado
    json_resp = json.loads(content.decode("utf-8", errors="replace"))

    if status_code == 200 and json_resp.get("type") == "success":
        # Sucesso: captura o "id" do usuário inserido
        usuario_id = json_resp.get("id")  # Exemplo: "154036"
        df_contrato.at[idx, "idRadUsuarios"] = usuario_id
        df_contrato.at[idx, "statusInsercaoRad"] = "sucesso"
        df_contrato.at[idx, "logRetornoRad"] = json_resp
    else:
        # Em caso de falha: registra os logs e garante que idRadUsuarios fique como None
        df_contrato.at[idx, "idRadUsuarios"] = None
        df_contrato.at[idx, "statusInsercaoRad"] = "falha"
        df_contrato.at[idx, "logRetornoRad"] = json_resp


200 OK
b'{"type":"success","message":"Registro inserido com sucesso!","id":"181284","atualiza_campos":[{"tipo":"s","campo":"online","valor":""},{"tipo":"i","campo":"id_cliente","valor":"117696"}]}'


### Retorno

In [52]:
# Remove colunas logRetorno e idGrupo
df_contrato.drop(columns=["logRetorno", "idGrupo"], inplace=True)

In [53]:
df_contrato.to_excel(source_output)

In [54]:
df_contrato

,Unnamed: 0,idClienteIXC,filial_id,idCidade,Endereco,Numero,Bairro,Cep,idPlanoIXC,idVencimentoIXC,Login,Senha,nomePlano,idContratoIXC,statusInsercao,idRadUsuarios,statusInsercaoRad,logRetornoRad
0,0,117696,35,1659,Vila São Sebastião,SN,Centro,57830-000,6824,25,teste1@radios.com,peixoto,100MB_SPEEDY,154047,sucesso,181284,sucesso,"{'type': 'success', 'message': 'Registro inser..."
